<a href="https://colab.research.google.com/github/ZaneZaiontz/YosAI/blob/main/YosAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TODO:**


*   List item
*   List item



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# with open('/content/drive/My Drive/YosAI/foo.txt', 'w') as f:
#     f.write('Hello Google Drive!')
# !cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

Imports

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


Step 1

In [ ]:
# Read, then decode for py2 compat.
text = open('/content/drive/My Drive/YosAI/haikuClean.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

FileNotFoundError: ignored

Step 2

Step 3

Step 4